In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
from tqdm import tqdm
#scraping imports:
from imutils import paths
import argparse
import requests
import cv2
import os
import ctypes
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import os
import glob
import time
import re
import base64

In [2]:
DATA = r"./data/top_anime_list/UsableAnimeList.xlsx"
df = pd.read_excel(DATA, sheet_name="usable")
anime_list = df["usable_title"].tolist()

In [3]:
class scrapeAnimeImageURLs:
    def __init__(self, anime_list):
        self.downloads_folder = r"C:\Users\kaush\Downloads"
        self.img_urls_dest = r"./data/anime_img_urls"
        self.options = webdriver.ChromeOptions()
        self.options.add_argument('--headless=new')
        self.driver = webdriver.Chrome(options=self.options)
        for anime in tqdm(anime_list, desc="AnimeList"):
            self.img_dest_folder = "./data/anime_images/" + anime + "/"
            self.get_image_URLS(anime)
            url_filename = self.move_download_file(anime)
            #even encoding = 'latin-1' will work
            raw_image_urls= pd.read_csv(url_filename, encoding='ISO-8859-1', header=None)
            raw_image_urls.columns = ["image_tag"]
            processed_image_urls = raw_image_urls["image_tag"].apply(self.get_url_from_html)
            self.get_image_from_url(processed_image_urls, anime, self.img_dest_folder)
        self.driver.quit()
    
    def get_image_URLS(self, anime_name):
        print(f"Currently scraping image urls for: {anime_name}")
        self.driver.get("https://www.google.com/search?tbm=isch&q=" + anime_name)
        for _ in range(1,9): #be mindful and increase the scroll range on each loop iteration
            self.driver.execute_script(f"window.scrollBy(0,{_*10000})") # so its roughly 100 images per 10000 y-axis scroll
            time.sleep(0.5) # of which a little less than half turn out to be useful
        self.driver.execute_script(r"""
            function saveImgTagToCSV(jsonArray) {
            const csvData = jsonArray.map((element) => {
                let imgTag = '';

                if (element.outerHTML) {
                imgTag = `"${element.outerHTML.replace(/"/g, '""')}"`;
                }

                return imgTag;
            }).join('\n');

            return csvData;
            }
            //then run this:
            const imgs_rg = Array.from(document.querySelectorAll('.rg_i'));
            const imgs_grn = Array.from(document.querySelectorAll('.GRN7Gc'));
            const imgs_ll = Array.from(document.querySelectorAll('.ll3EIMe'));
            const imgs = imgs_rg.concat(imgs_grn, imgs_ll);
            img_data = saveImgTagToCSV(imgs);
            window.open('data:text/csv;charset=utf-8,' + escape(img_data))""")
        
    def move_download_file(self, anime_name):
        time.sleep(1)
        file_list = glob.glob(os.path.sep.join([self.downloads_folder, "/*"]))
        latest_file = max(file_list, key=lambda x: os.stat(x).st_mtime)
        self.created_filename = self.img_urls_dest+"/"+anime_name+".csv"
        os.replace(latest_file, self.created_filename)
        return self.created_filename

    def get_url_from_html(self, html_code):   
        match = re.search(r'<img.*?(?:src|data-src)="(.*?)".*?>', html_code)
        if match:
            source = match.group(1)
            return source #tested that this takes care of all cases.
    
    def get_image_from_url(self, url_sources, anime_name, dest_folder):
        print(f"Currently downloading images for: {anime_name}")
        if not os.path.exists(dest_folder):
            os.makedirs(dest_folder)
        count = 0
        for source in url_sources:
            if source.startswith('data:image') and "base64" in source:
                match = re.search(r'src="data:image\/.*?;base64,(.*?)"', source)
                if match:
                    base64_string = match.group(1)
                    # Decode the base64 string
                    image_data = base64.b64decode(base64_string)
                    count += 1
                    image_name = dest_folder + anime_name + "_" + str(count).zfill(3) + ".jpg"
                    with open(image_name, 'wb') as f:
                        f.write(image_data)
            else:
                if "favicon" not in source:
                    response = requests.get(source)
                    # Check if the request was successful
                    if response.status_code == 200:
                        count += 1
                        image_name = dest_folder + anime_name + "_" + str(count).zfill(3) + ".jpg"
                        # Open the file in binary mode and write the image content
                        with open(image_name, 'wb') as file:
                            file.write(response.content)

In [4]:
scraper = scrapeAnimeImageURLs(["cowboy bepop", "naruto"])

AnimeList:   0%|          | 0/2 [00:00<?, ?it/s]

Currently scraping image urls for: cowboy bepop
Currently downloading images for: cowboy bepop


AnimeList:  50%|█████     | 1/2 [00:39<00:39, 39.05s/it]

Currently scraping image urls for: naruto
Currently downloading images for: naruto


AnimeList: 100%|██████████| 2/2 [01:14<00:00, 37.02s/it]


    There are two kinds of image URLs in the image URLs file:
        - base64 encoded image URLs
        - normal image URLs
        

    Image classes:
        rg_i Q4LuWd
        GRN7Gc
        ll3IMe.CBvy7c
    Somehow we always have 0 LL images, but that's a worry for later....I'll keep it there just in case it works :D